In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import json
import h2o

from datetime import datetime

from utils import get_data, Config, clean_url, make_clickable, add_columns

from tree import make_tree

In [3]:
# Load all data (except unknown/manual testing)
# Original Desktop Runs+Re-Runs were performed between 2024-02-05 and 2024-02-12
# Additional browsers + more than one response_id per parsing URL was run between 2024-02-16 and TBD
# Mobile runs performed between 2024-02-28 and TBD
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Browser".name != 'Unknown' and "Response".resp_type = 'basic'; -- AND "Result".created_at < '2024-02-19';
"""
df = get_data(Config(), initial_data)
df = add_columns(df)

Connecting to the PostgreSQL database...
Connection successful


In [4]:
df["raw_header"] = df["raw_header"].apply(bytes).apply(lambda x: json.loads(x.decode("utf-8"))).astype(str)
def create_test_id(row):
    return f'{row["test_name"]}_{row["relation_info"]}_{row["org_scheme"]}_{row["org_host"]}_{row["resp_scheme"]}_{row["resp_host"]}_{row["response_id"]}_{row["resp_type"]}'

df["browser_id"] = df["browser_id"].astype("category")
# Takes a while (500s+) (might be faster to already do it with postgres but not too important)
df["test_id"] = df.apply(create_test_id, axis=1)
df["test_id"] = df["test_id"].astype("category")

In [5]:
responses = """
SELECT * from "Response";
"""
responses = get_data(Config(), responses)

Connecting to the PostgreSQL database...
Connection successful


# Overview

In [6]:
# Both basic + parsing mode
# Each test should have results 5+ times (such that we can better reason about noise + majority voting makes sense)
# Basic was executed 20x+ on Linux, 5x on Mac
# Parsing was executed 5+ on Linux, 2x on Mac (currently executing a third time)
# (In addition, a couple of "repeat"-mode runs were performed in the beginning
df.groupby(["automation_mode", "browser_id", "name", "version", "os", "headless_mode"], observed=True)["id"].count().sort_values(ascending=False).to_frame().reset_index()

,automation_mode,browser_id,name,version,os,headless_mode,id
0,selenium,48,chrome,120,Ubuntu 22.04,headless-new,250944
1,selenium,49,firefox,121,Ubuntu 22.04,headless,250944
2,selenium,45,brave,v1.62.156 (121.0.6167.139),Ubuntu 22.04,headless-new,250944
3,selenium,44,edge,121,Ubuntu 22.04,headless-new,250944
4,selenium,43,firefox,122,Ubuntu 22.04,headless,250944
5,selenium,42,chrome,121,Ubuntu 22.04,headless-new,250944
6,selenium,47,chrome,122,Ubuntu 22.04,headless-new,248096
7,intent,3,brave,1.60.116,Android 11,real,68842
8,intent,2,firefox,119.1.1,Android 11,real,68751
9,selenium,46,safari,17.3,macOS 14.3,real,60984


# Error + Timeout Analysis
- Only timeout exist, no other systematic failures anymore
- Systematic timeout: TAO in Safari -> fixed
- Couple of random timeouts in other tests
  - More basic tests than parsing tests (even though less exist)
  - Code 302 more often than others
  - referrer-iframe, oac_*, framing_iframe most often

In [7]:
# Timeout (2), Fail (1), Not-run (3)
filtered_df = df.groupby(["browser"])["test_status"].filter(lambda x: x.nunique() >= 2)
display(df[["browser", "test_status"]].loc[df.index.isin(filtered_df.index)].groupby(["browser"])["test_status"].value_counts().to_frame())

count
browser                                            test_status        
brave Android 11 1.60.116 intent real              0             56778
                                                   2             12062
                                                   3                 2
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... 0            250943
                                                   2                 1
chrome Android 11 120.0.6099.44 intent real        0             51264
                                                   2              1016
chrome Ubuntu 22.04 120 selenium headless-new      0            250940
                                                   2                 4
chrome Ubuntu 22.04 122 selenium headless-new      0            248095
                                                   2                 1
edge Ubuntu 22.04 121 selenium headless-new        0            250883
                                                   2                61
firefox Android 11 119.1.1 intent real             0             58589
                                                   2             10150
                                                   3                12
firefox Ubuntu 22.04 121 selenium headless         0            250923
                                                   2                21
firefox Ubuntu 22.04 122 selenium headless         0            250920
                                                   2                24
firefox_beta Android 11 122.0b1 intent real        0             51736
                                                   2               369
                                                   3                 2
opera Android 11 78.4.4143.75735 intent real       0             54716
                                                   2              4936
                                                   3                 6
safari macOS 14.3 17.3 selenium real               0             60752
                                                   2               232

In [35]:
# Teststatus == 2 (timeout)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 2]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img/TAO (only Safari, fixed!)
# Others quite rare
with pd.option_context("display.max_rows", 80):
    display(df.loc[(df["test_status"] == 2)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
brave Android 11 1.60.116 intent real,12062
firefox Android 11 119.1.1 intent real,10150
opera Android 11 78.4.4143.75735 intent real,4936
chrome Android 11 120.0.6099.44 intent real,1016
firefox_beta Android 11 122.0b1 intent real,369
safari macOS 14.3 17.3 selenium real,232
edge Ubuntu 22.04 121 selenium headless-new,61
firefox Ubuntu 22.04 122 selenium headless,24
firefox Ubuntu 22.04 121 selenium headless,21


count
resp_type status_code test_name                     browser                                                  
basic     302         oac_iframe                    firefox Android 11 119.1.1 intent real               4200
                                                    brave Android 11 1.60.116 intent real                4200
          200         oac_iframe                    brave Android 11 1.60.116 intent real                2520
                                                    firefox Android 11 119.1.1 intent real               2478
                      referrer_iframe               brave Android 11 1.60.116 intent real                2240
                                                    firefox Android 11 119.1.1 intent real               2226
                      framing_embed                 opera Android 11 78.4.4143.75735 intent real         1449
                      framing_object                opera Android 11 78.4.4143.75735 intent real         1406
                      framing_iframe                opera Android 11 78.4.4143.75735 intent real         1305
          302         referrer_iframe               firefox Android 11 119.1.1 intent real               1120
                                                    brave Android 11 1.60.116 intent real                1120
          200         framing_embed                 brave Android 11 1.60.116 intent real                 496
                      framing_object                brave Android 11 1.60.116 intent real                 469
                      framing_iframe                brave Android 11 1.60.116 intent real                 433
                      framing_embed                 chrome Android 11 120.0.6099.44 intent real           286
                      framing_object                chrome Android 11 120.0.6099.44 intent real           275
                      framing_iframe                chrome Android 11 120.0.6099.44 intent real           259
          302         framing_embed                 opera Android 11 78.4.4143.75735 intent real          213
                      framing_object                opera Android 11 78.4.4143.75735 intent real          205
                      framing_embed                 brave Android 11 1.60.116 intent real                 196
                      framing_object                brave Android 11 1.60.116 intent real                 194
                      framing_iframe                opera Android 11 78.4.4143.75735 intent real          187
                                                    brave Android 11 1.60.116 intent real                 176
          200         referrer_iframe               safari macOS 14.3 17.3 selenium real                  118
                      framing_embed                 firefox_beta Android 11 122.0b1 intent real           107
                      framing_object                firefox_beta Android 11 122.0b1 intent real           106
                      framing_iframe                firefox_beta Android 11 122.0b1 intent real           104
                      subresourceloadingCORP_img    opera Android 11 78.4.4143.75735 intent real           92
          302         referrer_iframe               safari macOS 14.3 17.3 selenium real                   74
                      framing_embed                 chrome Android 11 120.0.6099.44 intent real            65
                      framing_object                chrome Android 11 120.0.6099.44 intent real            64
                      framing_iframe                chrome Android 11 120.0.6099.44 intent real            59
                      subresourceloadingCORP_img    opera Android 11 78.4.4143.75735 intent real           33
          200         fullscreen_iframe             opera Android 11 78.4.4143.75735 intent real           29
                      oac_iframe                    firefox_beta Android 11 122.0b1 intent real            28
                      framing_ifra

In [34]:
# Teststatus == 3 (not-run)

# Rare timeouts in all browsers; Safari some more than others
display(df[df["test_status"] == 3]["browser"].value_counts().to_frame())

# Mostly 302 redirects for basic (oac_iframe, oac_window.open), referrer_iframe 
# perfAPI_img/TAO (only Safari, fixed!)
# Others quite rare
display(df.loc[(df["test_status"] == 3)].groupby(["resp_type", "status_code", "test_name"])["browser"].value_counts().to_frame().sort_values(by="count", ascending=False))

,count
browser,
firefox Android 11 119.1.1 intent real,12
opera Android 11 78.4.4143.75735 intent real,6
firefox_beta Android 11 122.0b1 intent real,2
brave Android 11 1.60.116 intent real,2


count
resp_type status_code test_name             browser                                            
basic     200         upgradeHSTS_subdomain firefox Android 11 119.1.1 intent real            8
                                            opera Android 11 78.4.4143.75735 intent real      4
                      upgradeHSTS_direct    firefox Android 11 119.1.1 intent real            2
                      upgradeHSTS_subdomain firefox_beta Android 11 122.0b1 intent real       2
          302         upgradeHSTS_subdomain brave Android 11 1.60.116 intent real             2
                                            firefox Android 11 119.1.1 intent real            2
                                            opera Android 11 78.4.4143.75735 intent real      2

# General Test Statistics

In [9]:
df[["browser", "resp_type", "test_name", "relation_info", "response_id", "org_scheme", "org_host", "resp_scheme", "resp_host"]].drop_duplicates().groupby(["resp_type", "browser"]).count()

test_name  \
resp_type browser                                                         
basic     brave Android 11 1.60.116 intent real                   10457   
          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...      10456   
          chrome Android 11 120.0.6099.44 intent real             10456   
          chrome Ubuntu 22.04 120 selenium headless-new           10456   
          chrome Ubuntu 22.04 121 selenium headless-new           10456   
          chrome Ubuntu 22.04 122 selenium headless-new           10456   
          edge Ubuntu 22.04 121 selenium headless-new             10456   
          firefox Android 11 119.1.1 intent real                  10460   
          firefox Ubuntu 22.04 121 selenium headless              10456   
          firefox Ubuntu 22.04 122 selenium headless              10456   
          firefox_beta Android 11 122.0b1 intent real             10457   
          opera Android 11 78.4.4143.75735 intent real            10459   
          opera iPadOS 17.2 4.4.1 (0) intent real                    32   
          safari macOS 14.3 17.3 selenium real                    10456   

                                                              relation_info  \
resp_type browser                                                             
basic     brave Android 11 1.60.116 intent real                       10456   
          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...          10456   
          chrome Android 11 120.0.6099.44 intent real                 10456   
          chrome Ubuntu 22.04 120 selenium headless-new               10456   
          chrome Ubuntu 22.04 121 selenium headless-new               10456   
          chrome Ubuntu 22.04 122 selenium headless-new               10456   
          edge Ubuntu 22.04 121 selenium headless-new                 10456   
          firefox Android 11 119.1.1 intent real                      10456   
          firefox Ubuntu 22.04 121 selenium headless                  10456   
          firefox Ubuntu 22.04 122 selenium headless                  10456   
          firefox_beta Android 11 122.0b1 intent real                 10456   
          opera Android 11 78.4.4143.75735 intent real                10456   
          opera iPadOS 17.2 4.4.1 (0) intent real                        32   
          safari macOS 14.3 17.3 selenium real                        10456   

                                                              response_id  \
resp_type browser                                                           
basic     brave Android 11 1.60.116 intent real                     10457   
          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...        10456   
          chrome Android 11 120.0.6099.44 intent real               10456   
          chrome Ubuntu 22.04 120 selenium headless-new             10456   
          chrome Ubuntu 22.04 121 selenium headless-new             10456   
          chrome Ubuntu 22.04 122 selenium headless-new             10456   
          edge Ubuntu 22.04 121 selenium headless-new               10456   
          firefox Android 11 119.1.1 intent real                    10460   
          firefox Ubuntu 22.04 121 selenium headless                10456   
          firefox Ubuntu 22.04 122 selenium headless                10456   
          firefox_beta Android 11 122.0b1 intent real               10457   
          opera Android 11 78.4.4143.75735 intent real              10459   
          opera iPadOS 17.2 4.4.1 (0) intent real                      32   
          safari macOS 14.3 17.3 selenium real                      10456   

                                                              org_scheme  \
resp_type browser                                                          
basic     brave Android 11 1.60.116 intent real                    10457   
          brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s...       10456   
          chrome Android 11 120.0.6099.44 intent real      

In [10]:
168774 + 10456

179230

In [11]:
# Some browsers only used the more response_ids per page mode, thus these numbers don't say much
df.groupby("browser")["clean_url"].nunique()

browser
brave Android 11 1.60.116 intent real                                  364
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    364
chrome Android 11 120.0.6099.44 intent real                            364
chrome Ubuntu 22.04 120 selenium headless-new                          364
chrome Ubuntu 22.04 121 selenium headless-new                          364
chrome Ubuntu 22.04 122 selenium headless-new                          364
edge Ubuntu 22.04 121 selenium headless-new                            364
firefox Android 11 119.1.1 intent real                                 364
firefox Ubuntu 22.04 121 selenium headless                             364
firefox Ubuntu 22.04 122 selenium headless                             364
firefox_beta Android 11 122.0b1 intent real                            364
opera Android 11 78.4.4143.75735 intent real                           364
opera iPadOS 17.2 4.4.1 (0) intent real                                  1
safari macOS 14.3

In [12]:
# Number of response_ids tested (should be 44415)
# Many response_ids have more than one test (e.g., both iframe and window.open) thus this is not enough to show that all tests have results!
df.groupby("browser")["response_id"].nunique()

browser
brave Android 11 1.60.116 intent real                                  189
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    189
chrome Android 11 120.0.6099.44 intent real                            189
chrome Ubuntu 22.04 120 selenium headless-new                          189
chrome Ubuntu 22.04 121 selenium headless-new                          189
chrome Ubuntu 22.04 122 selenium headless-new                          189
edge Ubuntu 22.04 121 selenium headless-new                            189
firefox Android 11 119.1.1 intent real                                 189
firefox Ubuntu 22.04 121 selenium headless                             189
firefox Ubuntu 22.04 122 selenium headless                             189
firefox_beta Android 11 122.0b1 intent real                            189
opera Android 11 78.4.4143.75735 intent real                           189
opera iPadOS 17.2 4.4.1 (0) intent real                                  1
safari macOS 14.3

In [48]:
df.loc[df["test_status"] == 0].groupby("browser")["test_id"].nunique()

browser
brave Android 11 1.60.116 intent real                                  10188
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new    10456
chrome Android 11 120.0.6099.44 intent real                            10450
chrome Ubuntu 22.04 120 selenium headless-new                          10456
chrome Ubuntu 22.04 121 selenium headless-new                          10456
chrome Ubuntu 22.04 122 selenium headless-new                          10456
edge Ubuntu 22.04 121 selenium headless-new                            10456
firefox Android 11 119.1.1 intent real                                 10212
firefox Ubuntu 22.04 121 selenium headless                             10456
firefox Ubuntu 22.04 122 selenium headless                             10456
firefox_beta Android 11 122.0b1 intent real                            10456
opera Android 11 78.4.4143.75735 intent real                           10328
opera iPadOS 17.2 4.4.1 (0) intent real                             

In [14]:
# Make sure that all responses actually have responses
# (create_repeat.py assumes that at least one browser collected the results)

r1 = df.groupby("label")["response_id"].nunique().sort_values()
r2 = responses.loc[responses["resp_type"] != "debug"].groupby("label")["id"].nunique().sort_values()

pd.concat([r1, r2], axis=1)

,response_id,id
label,,
CORS,6.0,6
CSPvsXFO,7.0,1447
COOP,11.0,2667
COEP,11.0,2648
CORP,11.0,3368
XFO,11.0,4607
TAO,13.0,1614
HSTS,14.0,3338
OAC,14.0,1713


In [15]:
# Number of response_ids for each "group": resp_type, test_name, relation_info
df.groupby(["resp_type", "test_name", "relation_info"])["response_id"].nunique().to_frame()

response_id
resp_type test_name                     relation_info              
basic     accesswindow_direct           direct                   11
          fetch_GET                     credentials               6
                                        custom_headers            6
                                        simple                    6
          fetch_TEST                    custom_method             6
          framing_embed                 direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_iframe                direct                   35
                                        nested                   35
                                        sandbox                  35
          framing_object                direct                   35
                                        nested                   35
                                        sandbox                  35
          fullscreen_iframe             child                    24
                                        child_allow              24
                                        child_sandbox            24
                                        direct                   24
          imgloading_iframe             direct                   17
          oac_iframe                    direct                   14
                                        sandbox                  14
          oac_window.open               window.open              14
          perfAPI_img                   direct                   13
          referrer_iframe               iframe                   16
                                        window.open              16
          script_execution_iframe       direct                   17
                                        sandbox                  17
          subresourceloadingCOEP_img    direct                   11
                                        sandbox                  11
          subresourceloadingCORP_img    direct                   11
                                        nested                   11
                                        sandbox                  11
          subresourceloadingCORP_object direct                   11
          upgradeHSTS_direct            direct                   14
          upgradeHSTS_subdomain         subdomain                14

# Stability (Results after 2x with only one rseponse_id per parsing URL test)
- Basic tests:
    - 21/(10456x5) tests with different outcomes!
    - 13x firefox subresourceloadingCOEP_img (random behavior bug)
    - 6x timeout vs message 4(x edge, 2x brave)
    - 1x brave referrer different outcome
    - 1x safari COEP different outcome?
- Parsing tests:
    - 755/(168774x5) tests with different outcomes!
    - quite a lot! last time we only had 1 single test with a different outcome? (We used different versions of the browsers)
    - 3x accesswindow_direct (null vs accessible), 1x fullscreen_iframe (timeout vs message), 1x img_loading_iframe (timeout vs message)
    - All others (700+) upgradeHSTS_subdomain and upgradeHSTS_direct
        - Prevalence: `brave ~= edge > chrome >> firefox >> safari`
        - Hints at systematic issue: maybe the reset did not work or something similar?
        - Mostly timeout vs redirect=false; however some are also timeout vs redirect=false vs redirect=true!
        - Timeout maybe related to the HTTPS upgrade feature in chromium-based browsers which can take some time (>5s?)? (Non-Authoritative-Reason: HttpsUpgrades)
        - Unclear how redirect true vs false could happen? Maybe HSTS clearing did not work?
        - Manual testing always results in redirect=false for all tried clean_urls!

### Update: multiple response_ids per parsing URL test
- basic (123+)
    - mostly firefox subresourceloadingCOEP_img (random behavior bug)
    - some rare instances of other tests (timeouts)
- parsing (5000+)
    - fetch_* timeout issue in firefox (fixed)
    - subresourceloadingCOEP_img and imgloading_iframe timeout issues (other browsers) (probably fixed)
    - upgradeHSTS_subdomain + upgradeHSTS_direct
    - rare instances of other tests
- mostly timeouts?: Firefox fetch, other browsers subresourceloadingCOEP_img
- maybe we have too many tests per page/or the timeout is not high enough -> decreased the number of tests for some feature groups (now it seems better)
- (OT: if running `desktop_selenium.py` in the repeat mode, one has to increate `--timeout_task` (15*100 = 1500s > 1000) or decrease `--max_urls_until_restart` as we ran the repeat tests with a 3xTIMEOUT)

In [49]:
# Remove all tests that timed out (test_status == 0), each test has at least one result that did not time out in each browser
df_org = df
df = df.loc[df["test_status"] == 0]

In [17]:
for group_name, resp_type_group in df.groupby("resp_type"):
    #if group_name == "basic":
    #    continue
    print(group_name)
    stab = resp_type_group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser"])[["outcome_str"]].agg(["count", "nunique"])    # Tests with more than one outcome
    diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
    print(f"{len(diff_outcomes)} tests have different outcomes!")
    display(diff_outcomes.reset_index()["browser"].value_counts().to_frame())
    display(diff_outcomes.reset_index()[["test_name", "browser"]].value_counts().to_frame())

    diff_with_responses = diff_outcomes.reset_index().groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "browser"])["response_id"].agg(list).to_frame().reset_index()
    
    for id, (test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, browser, response_ids) in diff_with_responses.iterrows():
        # TODO: shortcut, only show the first result for each test_name/browser; results are usually always the same!
        print(test_name, response_ids)
        rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"].isin(response_ids)) & (df["browser"] == browser)]
        with pd.option_context("display.max_colwidth", 200):
            display(rows[["outcome_str", "browser"]].value_counts().to_frame())
            disp = rows.drop_duplicates(subset="outcome_str")[["browser", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
            disp = disp.style.format({'full_url': make_clickable})
            display(disp)
        # input("Continue!")

basic
1029 tests have different outcomes!


,count
browser,
opera Android 11 78.4.4143.75735 intent real,483
firefox Android 11 119.1.1 intent real,173
firefox_beta Android 11 122.0b1 intent real,135
brave Android 11 1.60.116 intent real,104
firefox Ubuntu 22.04 122 selenium headless,54
firefox Ubuntu 22.04 121 selenium headless,53
chrome Android 11 120.0.6099.44 intent real,11
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,6
chrome Ubuntu 22.04 120 selenium headless-new,3


,,count
"(test_name, )","(browser, )",
subresourceloadingCOEP_img,firefox_beta Android 11 122.0b1 intent real,91
framing_object,opera Android 11 78.4.4143.75735 intent real,84
subresourceloadingCOEP_img,firefox Android 11 119.1.1 intent real,82
framing_embed,opera Android 11 78.4.4143.75735 intent real,80
subresourceloadingCORP_img,opera Android 11 78.4.4143.75735 intent real,75
fetch_GET,opera Android 11 78.4.4143.75735 intent real,74
framing_iframe,opera Android 11 78.4.4143.75735 intent real,69
subresourceloadingCOEP_img,firefox Ubuntu 22.04 122 selenium headless,54
oac_window.open,brave Android 11 1.60.116 intent real,53


accesswindow_direct [81]


,,count
outcome_str,browser,
{'window.open.opener': 'null'},safari macOS 14.3 17.3 selenium real,6
"{'window.open.opener': 'object ""[object Window]""'}",safari macOS 14.3 17.3 selenium real,1


...

,,count
outcome_str,browser,
{'response.redirected': False},firefox_beta Android 11 122.0b1 intent real,7
Fetch timed out,firefox_beta Android 11 122.0b1 intent real,3


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
1935962,firefox_beta Android 11 122.0b1 intent real,Fetch timed out,"[['Strict-Transport-Security', 'max-age=0']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=138&last_id=138&scheme=http
1936525,firefox_beta Android 11 122.0b1 intent real,{'response.redirected': False},"[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http


upgradeHSTS_subdomain [137, 140]


,,count
outcome_str,browser,
{'response.redirected': True},opera Android 11 78.4.4143.75735 intent real,5
Fetch timed out,opera Android 11 78.4.4143.75735 intent real,3
{'response.redirected': False},opera Android 11 78.4.4143.75735 intent real,3


,browser,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
2057139,opera Android 11 78.4.4143.75735 intent real,Fetch timed out,"[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
2059462,opera Android 11 78.4.4143.75735 intent real,{'response.redirected': False},"[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=140&last_id=140&scheme=http
2110755,opera Android 11 78.4.4143.75735 intent real,{'response.redirected': True},"[['Strict-Transport-Security', 'max-age=20; includeSubDomains; preload']]",upgradeHSTS_subdomain,http,sub.headers.websec.saarland,https,subdomain,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=1&label=HSTS&first_id=137&last_id=137&scheme=http


## More stability?!
- How many outcomes did we collect per test in the different browsers? Also shows tests with only one outcome
- How many of them are different?

In [51]:
# How many outcomes did we collect for each test?!
# Should be between 5 and 20+ (less than 5 is concerning as we cannot do proper majority voting in such cases)

final_df = pd.DataFrame()

for group, d in df.groupby("browser"):
    # Calculate counts and unique counts
    aggs = ["count"] # ["count", "nunique"]
    res = d.groupby(["test_id"], observed=True)["outcome_str"].agg(aggs)
    # Get value counts for each combination of counts and unique counts
    counts = res.value_counts().to_frame()
    counts = counts.rename(columns={"count": group})

    only_once = res.loc[res["count"] == 1]
    less_than_five = res.loc[res["count"] < 5]
    if 1 < len(only_once) < 200:
        print(f"Tests with only one collected outcome: {group}")
        display(only_once.reset_index()[["test_id"]].value_counts().to_frame())

    final_df = pd.concat([final_df, counts], axis=1)

display(final_df)

Tests with only one collected outcome: brave Android 11 1.60.116 intent real


,count
test_id,
framing_object_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_4_basic,1
framing_embed_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_26_basic,1
framing_embed_nested_http_sub.headers.websec.saarland_https_headers.webappsec.eu_36_basic,1
framing_object_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_33_basic,1
framing_object_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_32_basic,1
...,...
framing_iframe_sandbox_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_5_basic,0
framing_iframe_sandbox_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_6_basic,0
framing_iframe_sandbox_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_7_basic,0


Tests with only one collected outcome: chrome Android 11 120.0.6099.44 intent real


,count
test_id,
framing_embed_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_29_basic,1
framing_embed_sandbox_http_sub.headers.websec.saarland_https_headers.websec.saarland_25_basic,1
framing_embed_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_28_basic,1
framing_embed_nested_https_sub.headers.websec.saarland_https_headers.webappsec.eu_26_basic,1
framing_iframe_nested_http_sub.headers.websec.saarland_https_sub.sub.headers.websec.saarland_25_basic,1
...,...
framing_iframe_sandbox_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_6_basic,0
framing_iframe_sandbox_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_7_basic,0
framing_iframe_sandbox_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_8_basic,0


Tests with only one collected outcome: opera iPadOS 17.2 4.4.1 (0) intent real


,count
test_id,
fetch_GET_simple_http_sub.headers.websec.saarland_http_headers.websec.saarland_84_basic,1
fetch_GET_credentials_http_sub.headers.websec.saarland_https_headers.webappsec.eu_84_basic,1
fetch_GET_custom_headers_http_sub.headers.websec.saarland_http_headers.webappsec.eu_84_basic,1
fetch_TEST_custom_method_http_sub.headers.websec.saarland_http_sub.sub.headers.websec.saarland_84_basic,1
fetch_GET_custom_headers_http_sub.headers.websec.saarland_https_sub.sub.headers.websec.saarland_84_basic,1
...,...
framing_iframe_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_23_basic,0
framing_iframe_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_24_basic,0
framing_iframe_sandbox_http_sub.headers.websec.saarland_https_headers.webappsec.eu_25_basic,0


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Android 11 119.1.1 intent real,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real,opera iPadOS 17.2 4.4.1 (0) intent real,safari macOS 14.3 17.3 selenium real
count,,,,,,,,,,,,,,
5,8646.0,NaN,9701.0,NaN,NaN,NaN,NaN,9740.0,NaN,NaN,10039.0,3419.0,NaN,3773.0
4,717.0,NaN,604.0,NaN,NaN,NaN,NaN,131.0,NaN,NaN,164.0,506.0,NaN,313.0
3,360.0,NaN,83.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,211.0,275.0,NaN,10.0
35,132.0,NaN,NaN,NaN,NaN,NaN,NaN,121.0,NaN,NaN,NaN,NaN,NaN,NaN
2,126.0,NaN,32.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,316.0,NaN,4.0
20,84.0,NaN,NaN,NaN,NaN,NaN,NaN,84.0,NaN,NaN,NaN,NaN,NaN,NaN
25,70.0,NaN,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN
1,24.0,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,219.0,32.0,NaN
75,12.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# Which tests have how many outcomes?

grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].count().reset_index()

with pd.option_context("display.max_rows", 256):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["outcome_str"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,236.000000,236.000000,236.000000
mean,15.420224,573.364407,8845.402542
std,9.287180,589.095075,11840.316253
min,1.000000,8.000000,8.000000
25%,5.000000,176.000000,1360.000000
50%,23.838309,288.000000,4863.500000
75%,24.000000,544.000000,8646.000000
max,24.000000,1680.000000,40320.000000


mean  \
browser                                            resp_type test_name                                  
firefox Ubuntu 22.04 122 selenium headless         basic     subresourceloadingCORP_img     24.000000   
                                                             subresourceloadingCORP_object  24.000000   
                                                             upgradeHSTS_direct             24.000000   
                                                             upgradeHSTS_subdomain          24.000000   
                                                             fullscreen_iframe              24.000000   
                                                             imgloading_iframe              24.000000   
                                                             perfAPI_img                    24.000000   
                                                             script_execution_iframe        24.000000   
                                                             subresourceloadingCOEP_img     24.000000   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     accesswindow_direct            24.000000   
                                                             fetch_GET                      24.000000   
                                                             fetch_TEST                     24.000000   
                                                             framing_embed                  24.000000   
                                                             framing_iframe                 24.000000   
                                                             framing_object                 24.000000   
firefox Ubuntu 22.04 121 selenium headless         basic     upgradeHSTS_direct             24.000000   
                                                             upgradeHSTS_subdomain          24.000000   
firefox Ubuntu 22.04 122 selenium headless         basic     accesswindow_direct            24.000000   
                                                             fetch_GET                      24.000000   
                                                             fetch_TEST                     24.000000   
                                                             framing_embed                  24.000000   
                                                             framing_iframe                 24.000000   
                                                             framing_object                 24.000000   
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) s... basic     fullscreen_iframe              24.000000   
                                                             imgloading_iframe              24.000000   
                                                             oac_iframe                     24.000000   
                                                             oac_window.open                24.000000   
                                                             perfAPI_img                    24.000000   
                                                             script_execution_iframe        24.000000   
                                                             subresourceloadingCOEP_img     24.000000   
                                                             subresourceloadingCORP_img     24.000000   
                                                             subresourceloadingCORP_object  24.000000   
                                                             upgradeHSTS_direct             24.000000   
                                                             upgradeHSTS_subdomain          24.000000   
firefox Ubuntu 22.04 121 selenium headless         basic     oac_iframe                     24.000000   
                                                             perfAPI_img                    24.000000   
                                                             script_execution_iframe        24.000000   
  

In [53]:
# Different outcomes (in percentage for a test group (test_name)
# (Does not take into account how often each test was executed)
grouped = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser", "resp_type"])['outcome_str'].nunique().reset_index()

grouped["diff_outcome"] = grouped["outcome_str"] > 1
with pd.option_context("display.max_rows", 67):
    res = grouped.groupby(["browser", "resp_type", "test_name"])["diff_outcome"].agg(["mean", "count", "sum"]).sort_values(by="mean", ascending=False)
    display(res.describe())
    display(res.loc[res["mean"] > 0])

,mean,count,sum
count,236.000000,236.000000,236.000000
mean,0.016226,573.364407,4.360169
std,0.049234,589.095075,15.007003
min,0.000000,8.000000,0.000000
25%,0.000000,176.000000,0.000000
50%,0.000000,288.000000,0.000000
75%,0.000000,544.000000,0.000000
max,0.285714,1680.000000,91.000000


,,,mean,count,sum
browser,resp_type,test_name,,,
opera Android 11 78.4.4143.75735 intent real,basic,upgradeHSTS_direct,0.285714,28,8
firefox_beta Android 11 122.0b1 intent real,basic,subresourceloadingCOEP_img,0.258523,352,91
opera Android 11 78.4.4143.75735 intent real,basic,fetch_GET,0.256944,288,74
brave Android 11 1.60.116 intent real,basic,oac_window.open,0.236607,224,53
firefox Android 11 119.1.1 intent real,basic,subresourceloadingCOEP_img,0.232955,352,82
opera Android 11 78.4.4143.75735 intent real,basic,fetch_TEST,0.218750,96,21
brave Android 11 1.60.116 intent real,basic,upgradeHSTS_direct,0.214286,28,6
opera Android 11 78.4.4143.75735 intent real,basic,subresourceloadingCORP_object,0.153409,176,27
firefox Ubuntu 22.04 122 selenium headless,basic,subresourceloadingCOEP_img,0.153409,352,54


In [54]:
# Run the test groups with many different outcomes again?
# Groups with non-negligble different outcomes are subresourceloadingCOEP_img, upgradeHSTS_subdomain, upgradeHSTS_direct, imgloading_iframe, fetch_TEST, fetch_GET
test_files_to_repeat =  ["fetch-cors.sub.html", "subresource-loading-coep.sub.html", "subresource-loading-csp.sub.html", "upgrade-hsts.sub.html"]

# TODO: implement utils.py/desktop_selenium mode to only rerun these files?!
# For now, simply run everything again to get some more results for stability?!

In [55]:
# Majority voting for the tests with multiple outcomes
df.sample(100).groupby(["test_name", "relation_info", "browser", "org_host", "resp_scheme", "org_origin", "resp_origin", "response_id"])["outcome_str"].agg([pd.Series.mode, "nunique", "count"]).reset_index().sort_values("count")

,test_name,relation_info,browser,org_host,resp_scheme,org_origin,resp_origin,response_id,mode,nunique,count
61,imgloading_iframe,direct,edge Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://headers.websec.saarland,111,error,1,1
73,referrer_iframe,iframe,firefox Ubuntu 22.04 122 selenium headless,sub.headers.websec.saarland,https,https://sub.headers.websec.saarland,https://sub.headers.websec.saarland,193,document.referrer: full_url,1,1
72,referrer_iframe,iframe,firefox Android 11 119.1.1 intent real,sub.headers.websec.saarland,http,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland,199,document.referrer: full_url,1,1
71,referrer_iframe,iframe,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) ...,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://sub.headers.websec.saarland,190,message timeout,1,1
70,perfAPI_img,direct,safari macOS 14.3 17.3 selenium real,sub.headers.websec.saarland,http,https://sub.headers.websec.saarland,http://headers.websec.saarland,211,{'requestStart != 0': False},1,1
...,...,...,...,...,...,...,...,...,...,...,...
29,framing_iframe,nested,chrome Ubuntu 22.04 121 selenium headless-new,sub.headers.websec.saarland,http,https://sub.headers.websec.saarland,http://sub.headers.websec.saarland,18,message timeout,1,1
28,framing_iframe,direct,firefox_beta Android 11 122.0b1 intent real,sub.headers.websec.saarland,http,https://sub.headers.websec.saarland,http://sub.headers.websec.saarland,39,message timeout,1,1
27,framing_iframe,direct,firefox Ubuntu 22.04 122 selenium headless,sub.headers.websec.saarland,http,https://sub.headers.websec.saarland,http://sub.headers.websec.saarland,37,message timeout,1,1
26,framing_iframe,direct,firefox Ubuntu 22.04 121 selenium headless,sub.headers.websec.saarland,https,http://sub.headers.websec.saarland,https://sub.sub.headers.websec.saarland,26,message timeout,1,1


# Browser differences!
- First perform majority voting
- Then diff the browsers

In [56]:
df = df_org
df = df.loc[df["test_status"] == 0]

In [57]:
df_stab = df

# Only keep one row for each test
# If the test had non-deterministic results, use majority voting! (only works reliably if there a enough repetitions of the tests runs, >= 5)

print("Original data entries", len(df))

# Majority voting (pd.Series.mode returns the most frequest item) (quite slow)

df = df.groupby(["test_id", "browser"], observed=True)["outcome_str"].agg(pd.Series.mode).reset_index()
print("Only one row for each test (per browser; majority voting)", len(df))

Original data entries 2087515
Only one row for each test (per browser; majority voting) 135314


In [58]:
# Merge back additional required properties!
# ["name", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "resp_type", "label", 'org_origin', 'resp_origin', 'status_code', 'raw_header']

df = df.merge(df_stab.drop_duplicates(subset=["test_id", "browser"]), on=["test_id", "browser"], how="left", suffixes=["", "_ignore"])


In [59]:
# If several values occur the same, the mode is an np.ndarray and not a string!
# What to do in such cases? If we have more repetitions the issue should probably go away automatically?
# If we have exactly 5 repetitions for each test, the issue cannot exist! (in the debug phase we sometimes have more or less than 5 repetitions and the issue can exist)
print("Tests with the same mode (highest frequency outcome):")
display(df.loc[df['outcome_str'].apply(lambda x: isinstance(x, np.ndarray))].groupby(["browser", "resp_type", "test_name"])["outcome_str"].count())

# In the rare cases, where it actually matters, we can just take the first? (this might be noise in the generated trees)
df['outcome_str'] = df['outcome_str'].apply(lambda x: x[0] if isinstance(x, np.ndarray) else x)

Tests with the same mode (highest frequency outcome):


browser                                       resp_type  test_name                 
brave Android 11 1.60.116 intent real         basic      upgradeHSTS_direct             1
firefox Android 11 119.1.1 intent real        basic      subresourceloadingCOEP_img     8
firefox_beta Android 11 122.0b1 intent real   basic      subresourceloadingCOEP_img     8
opera Android 11 78.4.4143.75735 intent real  basic      framing_embed                  8
                                                         framing_iframe                10
                                                         framing_object                10
                                                         subresourceloadingCORP_img     4
                                                         upgradeHSTS_direct             1
Name: outcome_str, dtype: int64

In [60]:
# Most tests have exactly one outcome, quite some have two, couple have three!
res = df.groupby(["resp_type", "test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "test_id"], observed=True)
res = res["outcome_str"].agg("nunique")
display(res.value_counts().to_frame())

,count
outcome_str,
1,8047
2,1798
3,412
4,199


In [61]:
# Basic:
# subresourceloadingCORP_object has different results for 100% of tests
# couple of others have 20%+ differences
# couple have 0%

# Parsing:
# oac_window.open has 99%
# Other test groups have between 0.5 - 7.5% of tests with different outcomes
r = res.reset_index()
r["diff"] = r["outcome_str"] != 1
r.groupby(["resp_type", "test_name", "relation_info"])["diff"].agg(["count", "sum", "mean"]).sort_values("mean", ascending=False)

count  sum      mean
resp_type test_name                     relation_info                       
basic     accesswindow_direct           direct            176  176  1.000000
          subresourceloadingCORP_object direct            176  176  1.000000
          referrer_iframe               window.open       256  224  0.875000
          oac_window.open               window.open       224  192  0.857143
          oac_iframe                    sandbox           224  144  0.642857
                                        direct            224  144  0.642857
          subresourceloadingCOEP_img    direct            176   87  0.494318
          fullscreen_iframe             direct            384  142  0.369792
          framing_embed                 direct            560  186  0.332143
          framing_object                direct            560  186  0.332143
          fullscreen_iframe             child_allow       384  113  0.294271
          perfAPI_img                   direct            208   55  0.264423
          framing_embed                 nested            560  120  0.214286
                                        sandbox           560  120  0.214286
          framing_object                nested            560  120  0.214286
                                        sandbox           560  120  0.214286
          script_execution_iframe       sandbox           272   21  0.077206
          upgradeHSTS_direct            direct             28    2  0.071429
          referrer_iframe               iframe            256   18  0.070312
          fullscreen_iframe             child             384   23  0.059896
          imgloading_iframe             direct            272   13  0.047794
          subresourceloadingCOEP_img    sandbox           176    4  0.022727
          framing_iframe                nested            560   11  0.019643
                                        sandbox           560   11  0.019643
          subresourceloadingCORP_img    direct            176    1  0.005682
          fetch_GET                     custom_headers     96    0  0.000000
                                        credentials        96    0  0.000000
          fullscreen_iframe             child_sandbox     384    0  0.000000
          fetch_TEST                    custom_method      96    0  0.000000
          fetch_GET                     simple             96    0  0.000000
          framing_iframe                direct            560    0  0.000000
          script_execution_iframe       direct            272    0  0.000000
          subresourceloadingCORP_img    nested            176    0  0.000000
                                        sandbox           176    0  0.000000
          upgradeHSTS_subdomain         subdomain          28    0  0.000000

In [62]:
# Select only test rows with more than one outcome
tests_with_mult_outcomes = r.loc[r["diff"]].set_index(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])
df_mult = df.loc[df[["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]].apply(tuple, axis=1).isin(tests_with_mult_outcomes.index)]
tests_with_more_than_one_outcome = df_mult.drop_duplicates(subset=["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])


all_tests_should_be =  df_org["test_id"].nunique() * df_org["browser"].nunique() # (num parsing tests + num_basic_tests) * num_browsers (168774+10456)
print(f"All test rows: {len(df)}, should be: {all_tests_should_be}, Rows with more than one outcome: {len(df_mult)}, Tests with more than one outcome: {len(tests_with_more_than_one_outcome)}")

All test rows: 135314, should be: 146440, Rows with more than one outcome: 31310, Tests with more than one outcome: 2409


In [63]:
# Display difference groups
for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    df_list = []
    for _, one_test in group.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"]):
        browsers = one_test.groupby("outcome_str")["browser"].unique().apply(list).apply(sorted).to_frame()
        new = True
        for df_b in df_list:
            if df_b.equals(browsers):
                new = False
        if new:
            df_list.append(browsers)
    print(len(df_list))
    for df_b in df_list:
        with pd.option_context("display.max_colwidth", 1000):
            display(df_b)
    #input("Continue!")

('accesswindow_direct', 'direct')
3


,browser
outcome_str,
"{'window.open.opener': 'object ""TypeError: Cannot read properties of null (reading \'opener\')""'}",[brave Android 11 1.60.116 intent real]
"{'window.open.opener': 'object ""TypeError: w is null""'}",[firefox Android 11 119.1.1 intent real]
"{'window.open.opener': 'object ""[object Window]""'}","[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real, opera Android 11 78.4.4143.75735 intent real, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new]"
"{'window.open.opener': 'object ""TypeError: Cannot read properties of null (reading \'opener\')""'}",[brave Android 11 1.60.116 intent real]
"{'window.open.opener': 'object ""TypeError: w is null""'}",[firefox Android 11 119.1.1 intent real]
"{'window.open.opener': 'object ""[object Window]""'}","[firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real, opera Android 11 78.4.4143.75735 intent real, safari macOS 14.3 17.3 selenium real]"


,browser
outcome_str,
{'window.open.opener': 'null'},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real, opera Android 11 78.4.4143.75735 intent real, safari macOS 14.3 17.3 selenium real]"
"{'window.open.opener': 'object ""TypeError: Cannot read properties of null (reading \'opener\')""'}",[brave Android 11 1.60.116 intent real]
"{'window.open.opener': 'object ""TypeError: w is null""'}",[firefox Android 11 119.1.1 intent real]


('framing_embed', 'direct')
1


...

,browser
outcome_str,
error,"[firefox Android 11 119.1.1 intent real, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"
load,"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, opera Android 11 78.4.4143.75735 intent real]"
timeout: no event fired,[safari macOS 14.3 17.3 selenium real]


,browser
outcome_str,
error,"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Android 11 119.1.1 intent real, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real, opera Android 11 78.4.4143.75735 intent real]"
timeout: no event fired,[safari macOS 14.3 17.3 selenium real]


('upgradeHSTS_direct', 'direct')
2


,browser
outcome_str,
Fetch timed out,"[brave Android 11 1.60.116 intent real, firefox Android 11 119.1.1 intent real, opera Android 11 78.4.4143.75735 intent real]"
{'response.redirected': False},"[brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real]"
{'response.redirected': True},[safari macOS 14.3 17.3 selenium real]


,browser
outcome_str,
Fetch timed out,[opera Android 11 78.4.4143.75735 intent real]
{'response.redirected': False},"[brave Android 11 1.60.116 intent real, brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new, chrome Android 11 120.0.6099.44 intent real, chrome Ubuntu 22.04 120 selenium headless-new, chrome Ubuntu 22.04 121 selenium headless-new, chrome Ubuntu 22.04 122 selenium headless-new, edge Ubuntu 22.04 121 selenium headless-new, firefox Android 11 119.1.1 intent real, firefox Ubuntu 22.04 121 selenium headless, firefox Ubuntu 22.04 122 selenium headless, firefox_beta Android 11 122.0b1 intent real, safari macOS 14.3 17.3 selenium real]"


In [65]:
# Other approach to list the difference groups
def get_uniques(df):
    browsers = f"{df['browser'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df_mult.groupby(["test_name", "relation_info"]):
    print(grouping)
    unique_outcomes = {}
    group.groupby(["browser"])["outcome_str"].value_counts(normalize=False).reset_index().groupby(["outcome_str", "count"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        display(d)
        print()

('accesswindow_direct', 'direct')


,"['firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real', 'opera Android 11 78.4.4143.75735 intent real', 'safari macOS 14.3 17.3 selenium real']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new']",['brave Android 11 1.60.116 intent real'],['firefox Android 11 119.1.1 intent real']
{'window.open.opener': 'null'},32.0,60.0,NaN,NaN
"{'window.open.opener': 'object ""[object Window]""'}",144.0,116.0,NaN,NaN
"{'window.open.opener': 'object ""TypeError: Cannot read properties of null (reading \'opener\')""'}",NaN,NaN,176.0,NaN
"{'window.open.opener': 'object ""TypeError: w is null""'}",NaN,NaN,NaN,176.0



('framing_embed', 'direct')


,"['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'firefox Android 11 119.1.1 intent real', 'firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real', 'opera Android 11 78.4.4143.75735 intent real']",['safari macOS 14.3 17.3 selenium real']
message send,186.0,NaN
message timeout,NaN,186.0



('framing_embed', 'nested')


...

,"['firefox Android 11 119.1.1 intent real', 'firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real', 'safari macOS 14.3 17.3 selenium real']","['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 78.4.4143.75735 intent real']"
error,1.0,NaN
load,NaN,1.0



('subresourceloadingCORP_object', 'direct')


,"['brave Android 11 1.60.116 intent real', 'brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'opera Android 11 78.4.4143.75735 intent real']","['firefox Android 11 119.1.1 intent real', 'firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real']",['safari macOS 14.3 17.3 selenium real']
error,44.0,76.0,NaN
load,132.0,100.0,NaN
timeout: no event fired,NaN,NaN,176.0



('upgradeHSTS_direct', 'direct')


,"['brave Android 11 1.60.116 intent real', 'firefox Android 11 119.1.1 intent real']",['opera Android 11 78.4.4143.75735 intent real'],"['brave Android 11 1.60.116 intent real', 'firefox Android 11 119.1.1 intent real', 'safari macOS 14.3 17.3 selenium real']","['brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new', 'chrome Android 11 120.0.6099.44 intent real', 'chrome Ubuntu 22.04 120 selenium headless-new', 'chrome Ubuntu 22.04 121 selenium headless-new', 'chrome Ubuntu 22.04 122 selenium headless-new', 'edge Ubuntu 22.04 121 selenium headless-new', 'firefox Ubuntu 22.04 121 selenium headless', 'firefox Ubuntu 22.04 122 selenium headless', 'firefox_beta Android 11 122.0b1 intent real']",['safari macOS 14.3 17.3 selenium real']
Fetch timed out,1.0,2.0,NaN,NaN,NaN
{'response.redirected': False},NaN,NaN,1.0,2.0,NaN
{'response.redirected': True},NaN,NaN,NaN,NaN,1.0


In [66]:
# Simmilarity between browsers and versions!

# Function to calculate absolute and percentage overlap
def calculate_overlap(df, col1, col2):
    df = df[[col1, col2]].dropna()
    df.columns.values[0] = 0
    df.columns.values[1] = 1
    absolute_overlap = (df[0] == df[1]).sum()
    unique_rows = len(df)
    diff_rows = unique_rows - absolute_overlap
    percentage_diff = (diff_rows / unique_rows) * 100
    return diff_rows, percentage_diff, unique_rows


def display_overlap(df, show_all=True, name="All"):
    print(name)
    sim_frame = df.set_index(["test_id", "browser"])["outcome_str"].unstack().astype("str")
    sim_frame = sim_frame.reset_index().drop(columns=["test_id"])
    # Create a dictionary to store results
    results = {}
    matrix = {}
    
    # Iterate through all pairs of columns
    for i, col1 in enumerate(sim_frame.columns):
        for col2 in sim_frame.columns:
            key = f"{col1}_vs_{col2}"
            results[key] = calculate_overlap(sim_frame, col1, col2)
            try:
                l = matrix[col1]
            except KeyError:
                l = {}
            # 0 for absolute count of differences, 1 for percentage
            l[col2] = results[key][0]
            matrix[col1] = l
            
    
    # Convert the results to a DataFrame for better visualization
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Absolute Diff', 'Percentage Diff', "Count"])
    
    # Display the results
    if show_all:
        display(results_df)
    
    # Display difference matrix
    with pd.option_context("display.max_columns", 28):
        if show_all:
            display(pd.DataFrame(matrix))
        else:
            display(pd.DataFrame(matrix).drop_duplicates().T)


display_overlap(df)

for name, group in df.groupby("test_name"):
    display_overlap(group, show_all=False, name=name)

All


,Absolute Diff,Percentage Diff,Count
brave Android 11 1.60.116 intent real_vs_brave Android 11 1.60.116 intent real,0,0.000000,10456
brave Android 11 1.60.116 intent real_vs_brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,879,8.406656,10456
brave Android 11 1.60.116 intent real_vs_chrome Android 11 120.0.6099.44 intent real,867,8.291890,10456
brave Android 11 1.60.116 intent real_vs_chrome Ubuntu 22.04 120 selenium headless-new,861,8.234507,10456
brave Android 11 1.60.116 intent real_vs_chrome Ubuntu 22.04 121 selenium headless-new,861,8.234507,10456
...,...,...,...
safari macOS 14.3 17.3 selenium real_vs_firefox Ubuntu 22.04 122 selenium headless,1119,10.701989,10456
safari macOS 14.3 17.3 selenium real_vs_firefox_beta Android 11 122.0b1 intent real,1155,11.046289,10456
safari macOS 14.3 17.3 selenium real_vs_opera Android 11 78.4.4143.75735 intent real,2003,19.156465,10456
safari macOS 14.3 17.3 selenium real_vs_opera iPadOS 17.2 4.4.1 (0) intent real,10424,99.693956,10456


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,chrome Android 11 120.0.6099.44 intent real,chrome Ubuntu 22.04 120 selenium headless-new,chrome Ubuntu 22.04 121 selenium headless-new,chrome Ubuntu 22.04 122 selenium headless-new,edge Ubuntu 22.04 121 selenium headless-new,firefox Android 11 119.1.1 intent real,firefox Ubuntu 22.04 121 selenium headless,firefox Ubuntu 22.04 122 selenium headless,firefox_beta Android 11 122.0b1 intent real,opera Android 11 78.4.4143.75735 intent real,opera iPadOS 17.2 4.4.1 (0) intent real,safari macOS 14.3 17.3 selenium real
brave Android 11 1.60.116 intent real,0,879,867,861,861,861,861,940,1584,1583,1607,1126,10156,2567
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,879,0,83,71,72,72,72,1583,1058,1057,1081,582,10424,2039
chrome Android 11 120.0.6099.44 intent real,867,83,0,12,13,13,13,1583,1016,1015,1039,528,10418,2010
chrome Ubuntu 22.04 120 selenium headless-new,861,71,12,0,1,1,1,1577,1016,1015,1039,528,10424,2010
chrome Ubuntu 22.04 121 selenium headless-new,861,72,13,1,0,0,0,1577,1017,1016,1040,529,10424,2011
chrome Ubuntu 22.04 122 selenium headless-new,861,72,13,1,0,0,0,1577,1017,1016,1040,529,10424,2011
edge Ubuntu 22.04 121 selenium headless-new,861,72,13,1,0,0,0,1577,1017,1016,1040,529,10424,2011
firefox Android 11 119.1.1 intent real,940,1583,1583,1577,1577,1577,1577,0,886,889,876,1760,10180,1967
firefox Ubuntu 22.04 121 selenium headless,1584,1058,1016,1016,1017,1017,1017,886,0,3,53,1100,10424,1120
firefox Ubuntu 22.04 122 selenium headless,1583,1057,1015,1015,1016,1016,1016,889,3,0,56,1099,10424,1119


accesswindow_direct


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,firefox Android 11 119.1.1 intent real,firefox Ubuntu 22.04 121 selenium headless
brave Android 11 1.60.116 intent real,0,176,176,176
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,176,0,176,28
chrome Android 11 120.0.6099.44 intent real,176,0,176,28
chrome Ubuntu 22.04 120 selenium headless-new,176,0,176,28
chrome Ubuntu 22.04 121 selenium headless-new,176,0,176,28
chrome Ubuntu 22.04 122 selenium headless-new,176,0,176,28
edge Ubuntu 22.04 121 selenium headless-new,176,0,176,28
firefox Android 11 119.1.1 intent real,176,176,0,176
firefox Ubuntu 22.04 121 selenium headless,176,28,176,0
firefox Ubuntu 22.04 122 selenium headless,176,28,176,0


...

,brave Android 11 1.60.116 intent real,firefox Android 11 119.1.1 intent real,safari macOS 14.3 17.3 selenium real
brave Android 11 1.60.116 intent real,0,32,176
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0,32,176
chrome Android 11 120.0.6099.44 intent real,0,32,176
chrome Ubuntu 22.04 120 selenium headless-new,0,32,176
chrome Ubuntu 22.04 121 selenium headless-new,0,32,176
chrome Ubuntu 22.04 122 selenium headless-new,0,32,176
edge Ubuntu 22.04 121 selenium headless-new,0,32,176
firefox Android 11 119.1.1 intent real,32,0,176
firefox Ubuntu 22.04 121 selenium headless,32,0,176
firefox Ubuntu 22.04 122 selenium headless,32,0,176


upgradeHSTS_direct


,brave Android 11 1.60.116 intent real,brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,opera Android 11 78.4.4143.75735 intent real,safari macOS 14.3 17.3 selenium real
brave Android 11 1.60.116 intent real,0,1,1,1
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,1,0,2,1
chrome Android 11 120.0.6099.44 intent real,1,0,2,1
chrome Ubuntu 22.04 120 selenium headless-new,1,0,2,1
chrome Ubuntu 22.04 121 selenium headless-new,1,0,2,1
chrome Ubuntu 22.04 122 selenium headless-new,1,0,2,1
edge Ubuntu 22.04 121 selenium headless-new,1,0,2,1
firefox Android 11 119.1.1 intent real,0,1,1,1
firefox Ubuntu 22.04 121 selenium headless,1,0,2,1
firefox Ubuntu 22.04 122 selenium headless,1,0,2,1


upgradeHSTS_subdomain


,brave Android 11 1.60.116 intent real
brave Android 11 1.60.116 intent real,0
brave Ubuntu 22.04 v1.62.156 (121.0.6167.139) selenium headless-new,0
chrome Android 11 120.0.6099.44 intent real,0
chrome Ubuntu 22.04 120 selenium headless-new,0
chrome Ubuntu 22.04 121 selenium headless-new,0
chrome Ubuntu 22.04 122 selenium headless-new,0
edge Ubuntu 22.04 121 selenium headless-new,0
firefox Android 11 119.1.1 intent real,0
firefox Ubuntu 22.04 121 selenium headless,0
firefox Ubuntu 22.04 122 selenium headless,0


In [67]:
# How many rows exist for tree creation
df[["test_name", "label", "relation_info"]].value_counts()

test_name                      label       relation_info 
fullscreen_iframe              PP          child_allow       4992
                                           child             4992
                                           direct            4992
                                           child_sandbox     4992
framing_embed                  CSP-FA      direct            3536
script_execution_iframe        CSP-SCRIPT  direct            3536
                                           sandbox           3536
imgloading_iframe              CSP-IMG     direct            3536
framing_object                 CSP-FA      direct            3536
framing_iframe                 CSP-FA      direct            3536
                                           sandbox           3534
framing_object                 CSP-FA      sandbox           3533
                                           nested            3532
framing_iframe                 CSP-FA      nested            3532
framing_embed     

In [227]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = df.groupby(["test_id"], observed=True)["outcome_str"].transform("nunique") != 1
tree_df = df.loc[condition]
print("Remove tests that are the same in all browsers", len(tree_df))
base_dir = f"trees/{datetime.today().strftime('%Y-%m-%dT%H:%M')}" 

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]
    group = group[["outcome_str", *pred_cols]]
    tree = make_tree(group, pred_cols, group_name, base_dir)
    # print(tree)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 29 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,100.0 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Remove tests that are the same in all browsers 67632
('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 29 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,100.0 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct'), datapoints: 572
('fetch_GET', 'credentials')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


...

Create tree: ('subresourceloadingCORP_object', 'direct'), datapoints: 1552
('upgradeHSTS_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 30 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_direct', 'direct'), datapoints: 2450
('upgradeHSTS_subdomain', 'subdomain')
Connecting to H2O server at http://localhost:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 days 1 hour 30 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,3 months and 22 days
H2O_cluster_name:,H2O_from_python_ubuntu_u7y7iq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.9 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('upgradeHSTS_subdomain', 'subdomain'), datapoints: 808


# Manual analysis

In [ ]:
# Chromium-based treats host-source as case sensitive even though it should not? (URLs and CSP is case-insensitive!)
# Play around: https://observer.sectec.rocks/opg/iframe/?url=https://echo.sectec.rocks/echo/?content-security-policy=img-src%20HTTPS://echo.sectec.rocks&ecocnt_css=%3Cimg%20src=https://echo.sectec.rocks/%3E&content-type=text/html
# Issue is in the scheme match which should be case-insensitive but is in Chromium-based browsers https://w3c.github.io/webappsec-csp/#match-schemes
# host-part matching was fixed here https://issues.chromium.org/issues/41412675, seems like they forgot fixing scheme-sensitivity 
df.loc[df["raw_header"].str.contains("img-src HTTPS://")][["clean_url", "raw_header", "outcome_str", "browser"]].value_counts().to_frame()

# Test improved repeat mode

In [113]:
def create_test_id(row):
    return f'{row["test_name"]}_{row["relation_info"]}_{row["org_scheme"]}_{row["org_host"]}_{row["resp_scheme"]}_{row["resp_host"]}_{row["response_id"]}_{row["resp_type"]}'

df["browser_id"] = df["browser_id"].astype("category")
# Takes a while (500s+) (might be faster to already do it with postgres but not too important)
df["test_id"] = df.apply(create_test_id, axis=1)
df["test_id"] = df["test_id"].astype("category")

/tmp/ipykernel_802996/2089415988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["browser_id"] = df["browser_id"].astype("category")
/tmp/ipykernel_802996/2089415988.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["test_id"] = df.apply(create_test_id, axis=1)
/tmp/ipykernel_802996/2089415988.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [144]:
# Luckily this is now instant
test_counts = df.groupby(["test_id"], observed=True)["browser_id"].value_counts()

In [156]:
tests_to_repeat = test_counts.loc[test_counts < 5].reset_index()
tests_to_repeat = tests_to_repeat.loc[tests_to_repeat["browser_id"] != 12]
tests_to_repeat.head()

,test_id,browser_id,count
4147,accesswindow_direct_direct_https_sub.headers.w...,46,3
4198,accesswindow_direct_direct_https_sub.headers.w...,46,3
4234,accesswindow_direct_direct_https_sub.headers.w...,46,3
4532,accesswindow_direct_direct_https_sub.headers.w...,46,3
4755,accesswindow_direct_direct_https_sub.headers.w...,46,3


In [157]:
tests_to_repeat[["browser_id", "count"]].value_counts()

browser_id  count
46          3        19879
            2            4
Name: count, dtype: int64

In [164]:
rep = tests_to_repeat.merge(df.drop_duplicates(subset=["test_id"]), on=["test_id"], how="left")

In [166]:
rep["full_url"].iloc[0]

'https://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?timeout=5&resp_type=parsing&browser_id=42&label=COOP&first_id=22971&last_id=22971&scheme=https'

In [32]:
tsv = pd.read_csv("browser_count.tsv", sep=" ")
tsv.sort_values(by=["browser_id", "count", "value_count"])

,browser_id,count,value_count
9,2,0,1281
3,2,1,133057
4,2,2,32561
6,2,3,1497
12,2,4,764
15,3,0,268
24,3,1,24
19,3,2,126
14,3,3,360
11,3,4,863
